<a href="https://colab.research.google.com/github/luca-arts/luca-arts.github.io/blob/main/knowledge/AI/AvatarCLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AvatarCLIP: Zero-Shot Text-Driven Generation and Animation of 3D Avatars

Authored by Fangzhou Hong, Mingyuan Zhang, Liang Pan, Zhongang Cai, Lei Yang, Ziwei Liu

This colab notebook provides a quick example of avatar shape sculpting and texture generation part described in our paper.

For the coarse shape generation and motion generation, please refer to our official Github repo (https://github.com/hongfz16/AvatarCLIP).

Please note that due to the GPU memory limitation, this demo cannot fully achieve the quality shown in the paper. To fit in the smaller GPU memory, we scale down the model, which harms its capacity. We also tune down the rendering resolution to reduce memory usage which will hurt the supervision from CLIP.

For more information, feel free to visit our project page (https://hongfz16.github.io/projects/AvatarCLIP.html).

The whole optimization process might take 4-10 hours depending on the machine you get allocated. Empirically, you can check the results after 10,000 iterations.

In [1]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

## Setup SMPL Models

Please register and download SMPL models here(https://smpl.is.tue.mpg.de/). Create folders under AvatarCLIP and put SMPL_NEUTRAL.pkl under the folder as follows.

```
./AvatarCLIP/
├── ...
└── smpl_models/
    └── smpl/
        └── SMPL_NEUTRAL.pkl
```

In [2]:
# @title Setup the Environment: 1. Common Dependencies
!nvidia-smi
!git clone https://github.com/hongfz16/AvatarCLIP.git
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r AvatarCLIP/requirements.txt
!git clone https://github.com/hongfz16/neural_renderer.git

Wed Jun  8 07:05:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Cloning into 'neural_renderer'...
remote: Enumerating objects: 1445, done.
remote: Counting objects: 100% (1445/1445), done.
remote: Compressing objects: 100% (492/492), done.
remote: Total 1445 (delta 937), reused 1440 (delta 935), pack-reused 0
Receiving objects: 100% (1445/1445), 26.21 MiB | 27.58 MiB/s, done.
Resolving deltas: 100% (937/937), done.


In [1]:
# @title Setup the Environment: 2. Modify neural_renderer
with open('neural_renderer/neural_renderer/perspective.py', 'r') as f:
  lines = f.readlines()
lines.insert(19, '    x[z<=0] = 0\n')
lines.insert(20, '    y[z<=0] = 0\n')
lines.insert(21, '    z[z<=0] = 0\n')
with open('neural_renderer/neural_renderer/perspective.py', 'w') as f:
  for l in lines:
    f.write(l)

In [2]:
# @title Setup the Environment: 3. Compile neural_renderer
%cd /content
%cd neural_renderer
!python3 setup.py install
%cd ../AvatarCLIP
%cd AvatarGen/AppearanceGen

/content
/content/neural_renderer
running install
running bdist_egg
running egg_info
writing neural_renderer_pytorch.egg-info/PKG-INFO
writing dependency_links to neural_renderer_pytorch.egg-info/dependency_links.txt
writing top-level names to neural_renderer_pytorch.egg-info/top_level.txt
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:352: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
adding license file 'LICENSE'
writing manifest file 'neural_renderer_pytorch.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying neural_renderer/perspective.py -> build/lib.linux-x86_64-3.7/neural_renderer
running build_ext
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/neural_renderer
copying build/lib.linux-x86_64-3.7/neural_rende

In [3]:
#####################################################################
# This code block is for myself to quickly setup SMPL models.       #
# Due to the license restriction, I cannot provide the SMPL models. #
# Please register and download at https://smpl.is.tue.mpg.de/       #
#####################################################################

%cd /content/AvatarCLIP/AvatarGen/AppearanceGen
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir -p ../../smpl_models/smpl
!cp '/content/gdrive/My Drive/AI/AvatarCLIP/smpl_models/smpl/models/basicmodel_neutral_lbs_10_207_0_v1.1.0.pkl' ../../smpl_models/smpl/SMPL_NEUTRAL.pkl

/content/AvatarCLIP/AvatarGen/AppearanceGen
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Important: Restart Runtime Before Proceeding (Menu => Runtime => Restart and run all)

This is for the self compiled module (neural_renderer) to be activated.

In [4]:
#@title Import Necessary Modules
%cd /content/AvatarCLIP/AvatarGen/AppearanceGen
import os
import time
import logging
import argparse
import random
import numpy as np
import cv2 as cv
import trimesh
import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from shutil import copyfile
from icecream import ic
from tqdm import tqdm
from pyhocon import ConfigFactory
from models.dataset import Dataset
from models.dataset import SMPL_Dataset
from models.fields import RenderingNetwork, SDFNetwork, SingleVarianceNetwork, NeRF
from models.renderer import NeuSRenderer
from models.utils import lookat, random_eye, random_at, render_one_batch, batch_rodrigues
from models.utils import sphere_coord, random_eye_normal, rgb2hsv, differentiable_histogram
from models.utils import my_lbs, readOBJ
import clip
from smplx import build_layer
import imageio

to8b = lambda x : (255*np.clip(x,0,1)).astype(np.uint8)
from main import Runner

/content/AvatarCLIP/AvatarGen/AppearanceGen


In [5]:
#@title Input Appearance Description (e.g. Iron Man)
AppearanceDescription = "An 80s guitar player with flames coming from his guitar" #@param {type:"string"}

torch.set_default_tensor_type('torch.cuda.FloatTensor')
FORMAT = "[%(filename)s:%(lineno)s - %(funcName)20s() ] %(message)s"
logging.basicConfig(level=logging.INFO, format=FORMAT)
conf_path = 'confs/examples_small/example.conf'
f = open(conf_path)
conf_text = f.read()
f.close()
conf_text = conf_text.replace('{TOREPLACE}', AppearanceDescription)
# print(conf_text)
conf = ConfigFactory.parse_string(conf_text)
print("Prompt: {}".format(conf.get_string('clip.prompt')))
print("Face Prompt: {}".format(conf.get_string('clip.face_prompt')))
print("Back Prompt: {}".format(conf.get_string('clip.back_prompt')))

Prompt: a 3D rendering of a An 80s guitar player with flames coming from his guitar in unreal engine
Face Prompt: a 3D rendering of the face of An 80s guitar player with flames coming from his guitar in unreal engine
Back Prompt: a 3D rendering of the back of An 80s guitar player with flames coming from his guitar in unreal engine


In [ ]:
#@title Start the Generation!
runner = Runner(conf_path, 'train_clip', 'smpl', False, True, conf)
runner.init_clip()
runner.init_smpl()
runner.train_clip()

Load data: Begin


[main.py:158 -             __init__() ] Load pretrain: ./pretrained_models/zero_beta_stand_pose_small.pth
[main.py:619 -        load_pretrain() ] End


Load data: End
Use head height: 0.7
Prompt: a 3D rendering of a An 80s guitar player with flames coming from his guitar in unreal engine
Face Prompt: a 3D rendering of the face of An 80s guitar player with flames coming from his guitar in unreal engine
Back Prompt: a 3D rendering of the back of An 80s guitar player with flames coming from his guitar in unreal engine


[utils.py:159 -    _init_num_threads() ] NumExpr defaulting to 2 threads.
  0%|          | 99/30000 [00:55<4:22:04,  1.90it/s]

./exp/smpl/example
iter:100 loss = 1.66093909740448 lr=9.900000000000001e-05
Validate: iter: 100, camera: 58


...

  5%|▍         | 1499/30000 [14:14<4:02:21,  1.96it/s]

./exp/smpl/example
iter:1500 loss = 1.4722838401794434 lr=0.0004986572021369778
Validate: iter: 1500, camera: 58
threshold: 0.0


[main.py:919 -        validate_mesh() ] End
  5%|▌         | 1599/30000 [15:23<4:00:22,  1.97it/s]

./exp/smpl/example
iter:1600 loss = 1.5831736326217651 lr=0.0004983752411181447
Validate: iter: 1600, camera: 58


  6%|▌         | 1684/30000 [16:09<4:00:07,  1.97it/s]

In [ ]:
#@title Generate a Video of Optimization Process (RGB)

import os
from tqdm import tqdm
import numpy as np
from IPython import display
from PIL import Image
from base64 import b64encode

image_folder = 'exp/smpl/example/validations_extra_fine'
image_fname = os.listdir(image_folder)
image_fname = sorted(image_fname)
image_fname = [os.path.join(image_folder, f) for f in image_fname]

init_frame = 1
last_frame = len(image_fname)
min_fps = 10
max_fps = 60
total_frames = last_frame - init_frame

length = 15 #Desired time of the video in seconds

frames = []
for i in range(init_frame, last_frame): #
    frames.append(Image.open(image_fname[i]))

#fps = last_frame/10
fps = np.clip(total_frames/length,min_fps,max_fps)

from subprocess import Popen, PIPE
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()
p.wait()
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

100%|██████████| 299/299 [00:03<00:00, 77.58it/s]


In [ ]:
#@title Generate a Video of Optimization Process (Normal)

import os
from tqdm import tqdm
import numpy as np
from IPython import display
from PIL import Image
from base64 import b64encode

image_folder = 'exp/smpl/example/normals'
image_fname = os.listdir(image_folder)
image_fname = sorted(image_fname)
image_fname = [os.path.join(image_folder, f) for f in image_fname]

init_frame = 1
last_frame = len(image_fname)
min_fps = 10
max_fps = 60
total_frames = last_frame - init_frame

length = 15 #Desired time of the video in seconds

frames = []
for i in range(init_frame, last_frame): #
    frames.append(Image.open(image_fname[i]))

#fps = last_frame/10
fps = np.clip(total_frames/length,min_fps,max_fps)

from subprocess import Popen, PIPE
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video_normal.mp4'], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()
p.wait()
mp4 = open('video_normal.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

100%|██████████| 299/299 [00:03<00:00, 90.44it/s]


#### Finally, don't forget to download the extracted meshes at `AvatarCLIP/AvatarGen/AppearanceGen/exp/smpl/example/meshes/*.ply`

In [ ]:
import os, shutil
mesh_path = '/content/AvatarCLIP/AvatarGen/AppearanceGen/exp/smpl/example/meshes/'
drive_mesh_path = os.path.join('/content/gdrive/MyDrive/AI/AvatarCLIP/Meshes/',AppearanceDescription.replace(' ','_'))
for i in os.listdir(mesh_path):
  _file = os.path.join(mesh_path,i)
  shutil.copy(_file,drive_mesh_path)
